In [1]:
import nasdaqdatalink
import os
import json
import quandl
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas_market_calendars as mcal
from datetime import datetime
import requests
from refresh_functions import *
#pd.options.display.float_format = '{:.3f}'.format

#store my API key
with open('C:/Users/meich/.nasdaq/data_link_apikey.json') as f:
    data=json.load(f)
    key=data['api_key']
quandl.ApiConfig.api_key = key

# Data Processing
-----

### Sharadar (SEP ~ Equity Prices)

for now, this cell should be run daily (except post-holiday, sundays, mondays)

must update code to pass list of dates between max(csv date) - daily date, in case miss a run

In [2]:
sharadarSEP()
sep = pd.read_csv('C:/Users/meich/CareerDocs/projects/stock_prediction/Data/SHARADAR_SEP.csv')

Data up to date
2022-12-27


### Tickers (filters)

In [3]:
tickers = sharadarTICKERS()

### Daily Metrics (filters)

In [4]:
sharadarDAILY()
daily = pd.read_csv('C:/Users/meich/CareerDocs/projects/stock_prediction/Data/SHARADAR_DAILY.csv')

# FILTER OUT STOCKS THAT WERE NEVER 500M MARKETCAP MINIMUM
daily.set_index('ticker',inplace=True)
daily['marketcap_max'] = daily.groupby('ticker').max()['marketcap']
daily = daily[daily['marketcap_max']>=500]
daily = daily.reset_index()

Data up to date
2022-12-27


### Short Interest Activity (Finra)

In [ ]:
finraSHORTS()

### Retail Trader Activity

In [6]:
# get historic values, process data
nasdaqRTAT()
rtat = pd.read_csv('C:/Users/meich/CareerDocs/projects/stock_prediction/Data/NDAQ_RTAT.csv')

Data up to date:
2022-12-27


In [7]:
# activity itself seems like it could be a good predictor of price drop (or high peak sentiment)
# rtat.sort_values('rtat_sentiment',ascending=False)

In [8]:
# CHECK FOR AUTO CORRELATION IN BOTH METRICS

In [9]:
# TEST FOR RMSE STRATEGY OF SIMILARITY INDEX USING ACT/SENT TO PRODUCE A SINGLE SCORE FOR AGG

In [10]:
# SET UP ACTIVITY TRIGGER TO CATCH DAILY VALUES AND SEND NOTIFICATION + VISUAL

### Create foundational dataset (shar)

In [11]:
combined = daily.merge(tickers,left_on='ticker',right_on='ticker',how='left')
combined = combined.merge(sep,left_on=['date','ticker'],right_on=['date','ticker'],how='left')
combined = combined.merge(rtat,left_on=['date','ticker'],right_on=['date','ticker'],how='left')

# Exploration/Analysis
-----

In [ ]:
# CREATE POTENTIALLY USEFUL FEATURES
rtat = fts_rtat(combined)

In [22]:
stock = combined[combined['ticker'] == 'TSLA'].copy()
stock = stock.sort_values(by='date')

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(x=stock['date'], y=stock['closeadj'], name="price"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=stock['date'], y=stock['rtat_activity']*100, name="activity"),
    secondary_y=True,
)

fig.add_trace(
    go.Scatter(x=stock['date'], y=stock['rtat_sentiment'], name="sentiment"),
    secondary_y=True,
)
fig.show()

### Start with a simple model -- Retail Activity Predictors & FWD Select Into

In [ ]:
# BUILD SIMPLE BASELINE MODEL TO PREDICT PRICE AT DIFFERENT TIME POINTS (EX: 5 DAY, 10 DAY, 20 DAY, 1 MONTH, 3 MONTH, 6 MONTH)
#       OR COULD CONSTRUCT AS A CLASSIFICATION MODEL TOO - PREDICT PROBABILITY THAT Y WILL FALL Y%

In [ ]:
# BUILD MODEL TO PREDICT STOCK PRICES
#    START THIS ONE WITH JUST SHORT INTEREST AND RETAIL TRADING ACTIVITY AND BUILD FROM THERE


In [7]:
# GET DAILY VALUES - USE MKT CALENDAR TO EXCLUDE WEEKEENDS/HOLIDAYS
nyse = mcal.get_calendar('NYSE')
lastdate = datetime.today().date() - pd.tseries.offsets.CustomBusinessDay(1, holidays = nyse.holidays().holidays)

RTAT = quandl.get_table('NDAQ/RTAT',date=lastdate,
                         paginate=True)

#RTAT[RTAT['activity']>0].sort_values('sentiment',ascending=False).head(20)

In [10]:
# NOTIFY (BASED ON MODELLING RESULTS ABOVE) + APPEND TO CSV
RTAT[RTAT['activity']>0].sort_values('activity',ascending=False).head(20)

,date,ticker,activity,sentiment
None,,,,
846,2022-12-23,TSLA,0.156500,-3
8131,2022-12-23,AAPL,0.043100,0
1307,2022-12-23,SPY,0.042800,3
1295,2022-12-23,SQQQ,0.032700,-5
7809,2022-12-23,AMZN,0.029100,0
2923,2022-12-23,NVDA,0.027800,-5
893,2022-12-23,TQQQ,0.024000,-5
2150,2022-12-23,QQQ,0.021400,3
7855,2022-12-23,AMD,0.021200,-6
